## Title: Customer Churn Prediction (MySQL + Logistic Regression)

Objective: Train a logistic regression model to predict customer churn, write predicted probabilities back to MySQL, and export a CSV for visualization (e.g., Tableau).

### 0. Environment & Configuration

"Set database credentials in a .env file (not tracked in git). See .env.example for structure."

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)  # IMPORTANT: allows replacing a previously-set value
import os

load_dotenv()  # reads .env if present

MYSQL_HOST = os.getenv("MYSQL_HOST", "127.0.0.1")
MYSQL_PORT = int(os.getenv("MYSQL_PORT", "3306"))
MYSQL_USER = os.getenv("MYSQL_USER", "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")  # keep secrets out of notebook
MYSQL_DB = os.getenv("MYSQL_DB", "churn_project")

In [ ]:
import pandas as pd
import mysql.connector as mc
import os
from dotenv import load_dotenv

load_dotenv(override=True)

QUERY = """
SELECT
  customerID AS id,
  is_monthly,
  auto_pay,
  CASE WHEN tenure < 12 THEN 1 ELSE 0 END AS short_tenure,
  churn
FROM customers_clean;
"""

conn = mc.connect(
    host=os.getenv("MYSQL_HOST","127.0.0.1"),
    port=int(os.getenv("MYSQL_PORT","3306")),
    user=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASSWORD"),
    database=os.getenv("MYSQL_DB","churn_project")
)
df = pd.read_sql(QUERY, conn)
conn.close()

df.head(10)

### 1) EDA sanity checks

Goal: quick data validation from customers_clean.

In [ ]:
import pandas as pd, numpy as np

print(df.shape)
display(df.head(3))

# Basic label balance
df['churn'].value_counts().rename_axis('label').to_frame('count')
df['churn'].value_counts(normalize=True).mul(100).round(2).rename('pct')

# Leakage check: make sure target not in features
set(df.columns) - {'id','is_monthly','auto_pay','short_tenure','churn'}


### 2) Baseline logistic regression

Goal: simple baseline using the three engineered features we already have.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

X = df[['is_monthly','auto_pay','short_tenure']].astype(float)
y = df['churn'].astype(int)
ids = df['id'].astype(str)

X_tr, X_te, y_tr, y_te, id_tr, id_te = train_test_split(
    X, y, ids, test_size=0.25, random_state=42, stratify=y
)

pipe = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('logreg', LogisticRegression(max_iter=2000, solver='lbfgs', class_weight=None))
]).fit(X_tr, y_tr)

p_tr = pipe.predict_proba(X_tr)[:,1]
p_te = pipe.predict_proba(X_te)[:,1]

print(f"AUC train: {roc_auc_score(y_tr, p_tr):.3f}")
print(f"AUC test : {roc_auc_score(y_te, p_te):.3f}")

### Step 3: One-hot encode categoricals
Goal: lift performance by adding SQL columns and one-hot in sklearn.

Categorical features like contract type and payment method can’t be used directly in regression models, so we convert them into binary indicators using one-hot encoding. This allows the model to capture differences across categories without assuming any numeric order. Adding these features should improve performance compared with the simple baseline.


In [ ]:
QUERY_PLUS = """
SELECT
  customerID AS id,
  is_monthly, auto_pay,
  CASE WHEN tenure < 12 THEN 1 ELSE 0 END AS short_tenure,
  Contract, PaymentMethod, gender, age_group,
  tenure, MonthlyCharges,
  churn
FROM customers_clean;
"""
conn = mc.connect(
    host=os.getenv("MYSQL_HOST"), port=int(os.getenv("MYSQL_PORT")),
    user=os.getenv("MYSQL_USER"), password=os.getenv("MYSQL_PASSWORD"),
    database=os.getenv("MYSQL_DB")
)
df2 = pd.read_sql(QUERY_PLUS, conn); conn.close()


### Step 4: ROC, Precision-Recall, and Classification Report

To understand model performance beyond accuracy, we use ROC and Precision-Recall curves. ROC shows how well the model separates churners from non-churners at different thresholds, summarized by AUC. Precision-Recall is especially useful with imbalanced data, highlighting tradeoffs between catching churners (recall) and avoiding false alarms (precision).


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, classification_report

# ROC
fpr, tpr, _ = roc_curve(y_te, p_te)
roc_auc = auc(fpr, tpr)
plt.figure(); plt.plot(fpr,tpr,label=f"AUC={roc_auc:.3f}"); plt.plot([0,1],[0,1],'--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC"); plt.legend(); plt.show()

# PR
prec, rec, _ = precision_recall_curve(y_te, p_te)
ap = average_precision_score(y_te, p_te)
plt.figure(); plt.plot(rec, prec, label=f"AP={ap:.3f}")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall"); plt.legend(); plt.show()

# Default 0.5 report
y_hat = (p_te >= 0.5).astype(int)
print(classification_report(y_te, y_hat, digits=3))



### Step 5: Threshold Tuning and Confusion Matrix
After choosing a threshold, we can visualize the confusion matrix to better understand errors.  
The matrix shows true negatives, false positives, false negatives, and true positives, giving a clear picture of model tradeoffs at the chosen cutoff.


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

def pick_threshold(p, y, target_recall=0.70):
    prec, rec, thr = precision_recall_curve(y, p)
    # thr has len-1 vs prec/rec; align
    thr = np.r_[0, thr]
    idx = np.where(rec >= target_recall)[0][-1] if np.any(rec >= target_recall) else np.argmax(rec)
    return float(thr[idx]), float(prec[idx]), float(rec[idx])

thr, pr_at_thr, rc_at_thr = pick_threshold(p_te, y_te, target_recall=0.70)
print(f"Chosen threshold: {thr:.3f} (precision={pr_at_thr:.3f}, recall={rc_at_thr:.3f})")

y_hat = (p_te >= thr).astype(int)
tn, fp, fn, tp = confusion_matrix(y_te, y_hat).ravel()
print({"tp":tp,"fp":fp,"fn":fn,"tn":tn})


In [ ]:
import seaborn as sns
import numpy as np

cm = confusion_matrix(y_te, y_hat)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Predicted: No churn", "Predicted: Churn"],
            yticklabels=["Actual: No churn", "Actual: Churn"])
plt.title(f"Confusion Matrix (threshold={thr:.2f})")
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()